In [ ]:
#export
from local.torch_basics import *
from local.test import *
from local.core import *
from local.layers import *
from local.data.all import *
from local.notebook.showdoc import show_doc
from local.optimizer import *
from local.learner import *
from local.metrics import *
from local.text.core import *
from local.text.data import *
from local.text.models.core import *
from local.text.models.awdlstm import *
from local.text.learner import *
from local.callback.rnn import *
from local.callback.all import *

# ULMFiT

## Finetune a pretrained Language Model

First we get our data and tokenize it.

In [ ]:
path = untar_data(URLs.IMDB)
tokenize_folder(path, folders=['train', 'test', 'unsup'])

In [ ]:
path = untar_data(URLs.IMDB).parent/'imdb_tok'

In [ ]:
count = pickle.load(open(path/'counter.pkl', 'rb'))
vocab = make_vocab(count)

In [ ]:
texts = get_files(path, extensions=['.txt'])

In [ ]:
lens = [int(f.with_suffix('.len').read()) for f in texts]

In [ ]:
len(texts),len(lens)

(100000, 100000)

Then we put it in a `DataSource`. For a language model, we don't have targets, so there is only one transform to numericalize the texts. Note that `tokenize_df` returns the count of the words in the corpus to make it easy to create a vocabulary.

In [ ]:
def read_file(f): return L(f.read().split(' '))

In [ ]:
splits = RandomSplitter()(texts)
vocab = make_vocab(count)
dsrc = DataSource(texts, [[read_file, Numericalize(vocab)]], filts=splits)

Then we use that `DataSource` to create a `DataBunch`. Here the class of `TfmdDL` we need to use is `LMDataLoader` which will concatenate all the texts in a source (with a shuffle at each epoch for the training set), split it in `bs` chunks then read continuously through it.

In [ ]:
bs,sl=256,80
dbunch_lm = LMDataLoader.dbunchify(dsrc, lens=lens, bs=bs, seq_len=sl, val_bs=bs)

In [ ]:
dbunch_lm.show_batch()

Then we have a convenience method to directly grab a `Learner` from it, using the `AWD_LSTM` architecture.

In [ ]:
opt_func = partial(Adam, wd=0.1)
learn = language_model_learner(dbunch_lm, AWD_LSTM, vocab, opt_func=opt_func, metrics=[accuracy, Perplexity()], path=path)
learn.to_fp16(clip=0.1)

In [ ]:
learn.fit_one_cycle(1, 2e-2, moms=(0.8,0.7,0.8))

In [ ]:
learn.save('stage1')

In [ ]:
learn.load('stage1')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3, moms=(0.8,0.7,0.8))

Once we have fine-tuned the pretrained language model to this corpus, we save the encoder since we will use it for the classifier.

In [ ]:
learn.save_encoder('finetuned')

## Use it to train a classifier

In [ ]:
texts = get_files(path, extensions=['.txt'], folders=['train', 'test'])

In [ ]:
splits = GrandparentSplitter(valid_name='test')(texts)

For classification, we need to use two set of transforms: one to numericalize the texts and the other to encode the labels as categories.

In [ ]:
dsrc = DataSource(texts, [[read_file, Numericalize(vocab)], [parent_label, Categorize()]], filts=splits)

In [ ]:
lens = [int(f.with_suffix('.len').read()) for f in texts]

In [ ]:
res = [L(lens, use_list=None)[f] for f in dsrc.filts]

In [ ]:
bs = 64

In [ ]:
trn_dl = SortedDL(dsrc.train, res=res[0], create_batch=pad_collate, after_batch=[Cuda], shuffle=True, drop_last=True, bs=bs)
val_dl = SortedDL(dsrc.valid, res=res[1], create_batch=pad_collate, after_batch=[Cuda], bs=bs)
dbunch = DataBunch(trn_dl, val_dl)

In [ ]:
dbunch.show_batch(max_n=2)

Then we once again have a convenience function to create a classifier from this `DataBunch` with the `AWD_LSTM` architecture.

In [ ]:
opt_func = partial(Adam, wd=0.1)
learn = text_classifier_learner(dbunch, AWD_LSTM, vocab, metrics=[accuracy], path=path, drop_mult=0.5, opt_func=opt_func)

We load our pretrained encoder.

In [ ]:
learn = learn.load_encoder('finetuned')

Then we can train with gradual unfreezing and differential learning rates.

In [ ]:
lr = 1e-1 * bs/128

In [ ]:
learn.fit_one_cycle(1, lr, moms=(0.8,0.7,0.8), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.432456,0.216977,0.914560,01:23


In [ ]:
learn.freeze_to(-2)
lr /= 2
learn.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7,0.8), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.270958,0.182856,0.931280,01:38


In [ ]:
learn.freeze_to(-3)
lr /= 2
learn.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7,0.8), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.208677,0.169175,0.934000,02:14


In [ ]:
learn.unfreeze()
lr /= 5
learn.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7,0.8), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,0.151267,0.176616,0.935760,02:46
